In [50]:
import numpy as np
import pickle

In [63]:
file_name ='all tasks, ego V density 0.03, time length 60000, mode pso.pkl'
with open(file_name,'rb') as file:
    logs  = pickle.loads(file.read())
print(len(logs))
file_name ='finished tasks, ego V density 0.03, time length 60000, mode pso.pkl'
with open(file_name,'rb') as file:
    finsih_logs  = pickle.loads(file.read())
print(len(finsih_logs))

1774
4415


In [64]:
max_time = 0
for log in finsih_logs:
    max_time = max(max_time,log.total_time)
    print(log.total_time)
print('max time', max_time)

105
105
140
118
192
264
196
78
223
60
295
325
381
294
72
415
429
332
335
428
200
360
220
392
160
173
563
545
340
356
546
293
436
708
456
70
761
223
622
633
847
296
913
386
217
944
383
393
251
192
457
216
460
167
511
180
180
387
159
159
58
326
231
207
207
154
155
180
162
228
253
272
391
306
354
381
386
356
396
505
621
448
64
515
642
546
578
456
457
241
536
504
365
762
685
437
585
345
66
497
866
827
499
749
416
904
921
765
938
50
804
699
190
617
488
254
674
334
707
943
577
1049
1054
990
708
371
805
269
202
1170
803
809
794
306
313
236
261
836
532
281
813
183
312
507
900
1346
278
720
983
1096
265
271
1553
318
300
972
362
262
386
90
487
257
775
309
293
380
358
395
395
1868
680
686
345
261
551
820
464
609
831
619
854
435
247
637
433
948
362
52
768
548
564
640
782
1934
690
1074
874
673
204
543
721
909
245
248
54
287
2373
580
1055
1039
669
324
1135
693
934
308
1165
316
982
1252
311
357
1267
289
344
1067
494
1141
1566
913
586
557
95
1451
601
1754
230
804
663
478
691
486
162
149
1563
557
1582
1

In [65]:
max_time = 0
for log in finsih_logs:
    max_time = max(max_time,log.total_time)
    print(log.total_time)
print('max time', max_time)

105
105
140
118
192
264
196
78
223
60
295
325
381
294
72
415
429
332
335
428
200
360
220
392
160
173
563
545
340
356
546
293
436
708
456
70
761
223
622
633
847
296
913
386
217
944
383
393
251
192
457
216
460
167
511
180
180
387
159
159
58
326
231
207
207
154
155
180
162
228
253
272
391
306
354
381
386
356
396
505
621
448
64
515
642
546
578
456
457
241
536
504
365
762
685
437
585
345
66
497
866
827
499
749
416
904
921
765
938
50
804
699
190
617
488
254
674
334
707
943
577
1049
1054
990
708
371
805
269
202
1170
803
809
794
306
313
236
261
836
532
281
813
183
312
507
900
1346
278
720
983
1096
265
271
1553
318
300
972
362
262
386
90
487
257
775
309
293
380
358
395
395
1868
680
686
345
261
551
820
464
609
831
619
854
435
247
637
433
948
362
52
768
548
564
640
782
1934
690
1074
874
673
204
543
721
909
245
248
54
287
2373
580
1055
1039
669
324
1135
693
934
308
1165
316
982
1252
311
357
1267
289
344
1067
494
1141
1566
913
586
557
95
1451
601
1754
230
804
663
478
691
486
162
149
1563
557
1582
1

In [56]:
vars(log)

{'tid': 26,
 'vid': 14,
 'start_time': 962,
 'end_time': 1156,
 'total_time': 197,
 'time_vehicle_compute': 76,
 'time_ul_transmit': 16,
 'time_dl_transmit': 105,
 'remain_compute_size': 0.0,
 'task_size': 79,
 'remain_size_dl': 0.0,
 'result_size': 24,
 'remain_size_ul': 0.0,
 'generated_vid': 0,
 'experienced_resource_allocation': 10.0}

In [62]:
max_left_time = 0
for log in finsih_logs:
    time_left = log.total_time -log.time_dl_transmit
    max_left_time = max(max_left_time,time_left)
    print(time_left)
print('max_left_time',max_left_time)

44
44
80
28
99
62
99
57
90
36
112
48
55
91
59
98
120
125
114
97
109
93
106
106
94
107
182
116
154
161
167
200
254
274
306
66
327
188
343
387
383
261
411
363
191
464
351
361
225
165
397
189
448
136
479
149
149
361
113
113
34
306
200
89
89
74
131
161
148
74
99
79
89
78
47
89
91
95
103
139
89
125
49
55
165
57
61
85
85
53
53
59
53
229
155
195
99
89
45
165
200
134
191
124
175
226
154
164
177
41
92
161
125
243
160
99
206
212
154
251
220
186
245
206
270
255
252
211
89
290
315
330
358
248
255
123
196
309
230
179
268
75
55
350
375
225
112
425
287
292
142
99
188
131
100
99
87
43
59
61
36
91
94
100
96
112
86
127
70
171
122
128
76
70
65
138
106
110
102
136
172
92
136
168
148
147
117
35
145
121
103
160
202
184
154
262
165
173
76
150
100
193
81
81
41
201
344
210
227
305
343
70
372
394
223
183
394
129
307
176
102
133
370
140
191
265
81
376
615
475
220
240
85
547
252
795
107
730
389
288
618
291
135
120
858
326
855
745
184
356
351
453
553
143
217
921
142
216
138
82
158
473
161
182
563
149
281
286
142
2

In [31]:
print(log)

In [35]:
for log in logs:
    print(log.end_time)

1148
2046
2747
4758
5392
7399
8409
8970
9085
9714
9753
9956
10375
12261
12348
12452
12891
12906
14051
15481
15952
16132
16683
16741
16802
17034
17254
18298
18941
19037
19165
19676
19734
19735
20569
20671
20764
20766
22685
23441
23606
25076
25643
27128
27193
27656
28356
28371
30682
31388
31448
31544
31816
32144
32641
32786
33496
33647
33796
34509
35330
36365
36912
37094
37555
37882
39029
39558
39650
40077
40139
40330
40374
40394
40706
40750
41237
41607
41761
43697
44160
44223
45066
45126
45520
45762
46585
46675
46750
47368
47472
47911
49046
49204
49710
49791
50285
51182
53474
54646
54814
55521
56043
56711
56788
56898
57037
57100
57192
57773
59148
59601
59761
